In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import scanpy as sc
import numpy as np
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupShuffleSplit, StratifiedGroupKFold
import seaborn as sns
from pathlib import Path

In [ ]:
base_dir = Path().resolve()
parent_dir = base_dir.parent
parent_dir

## Read flux and TMA preprocessed Data

### Read Metabolic Task data

In [ ]:
metabolic_task_df = pd.read_csv(parent_dir / 'data/scCellFie/scCellFie_metabolic_tasks_with_name.csv', low_memory=False)
metabolic_task_df.rename(columns={'Unnamed: 0':'cell'},inplace=True)
metabolic_task_df.head()

In [ ]:
print("count of +inf:", np.isposinf(metabolic_task_df.iloc[:,1:].to_numpy()).sum())
print("count of -inf:", np.isneginf(metabolic_task_df.iloc[:,1:].to_numpy()).sum())

In [ ]:
metabolic_task_df = metabolic_task_df.replace(-np.inf, 0)

### Read Reactions data

In [ ]:
reactions_df = pd.read_csv(parent_dir / 'data/scCellFie/scCellFie_reactions_with_name.csv', low_memory=False)
reactions_df.rename(columns={'Unnamed: 0':'cell'},inplace=True)
reactions_df.head()

In [ ]:
print("count of +inf:", np.isposinf(reactions_df.iloc[:,1:].to_numpy()).sum())
print("count of -inf:", np.isneginf(reactions_df.iloc[:,1:].to_numpy()).sum())

In [ ]:
reactions_df = reactions_df.replace(-np.inf, 0)

### Read selected gene expression data

In [ ]:
gene_df = pd.read_csv(parent_dir / 'data/scCellFie/scCellFie_genes_with_name.csv', low_memory=False)
gene_df.rename(columns={'Unnamed: 0':'cell'},inplace=True)
gene_df.head()

In [ ]:
print("count of +inf:", np.isposinf(gene_df.iloc[:,1:].to_numpy()).sum())
print("count of -inf:", np.isneginf(gene_df.iloc[:,1:].to_numpy()).sum())

### Read adata data

In [ ]:
adata = sc.read(parent_dir / 'data/h5ad/merged_TMA_processed_compatible.h5ad')
adata

In [ ]:
groups = adata.obs['Subject_ID']

In [ ]:
# Extract UMAP
umap_df = pd.DataFrame(
    adata.obsm['X_umap'],
    index=adata.obs.index,
    columns=['UMAP1', 'UMAP2']
)

# Extract spatial
spatial_df = pd.DataFrame(
    adata.obsm['spatial'],
    index=adata.obs.index,
    columns=['X_spatial', 'Y_spatial']
)

# Concatenate along columns
coords_df = pd.concat([umap_df, spatial_df], axis=1)
coords_df = coords_df.reset_index().rename(columns={"index": "cell"})
coords_df

In [ ]:
cols = ["Treatment_Status", "Subject_ID", "sample", "leiden" ]

obs_df = adata.obs[cols].copy()
obs_df = adata.obs[cols].reset_index().rename(columns={"index": "cell"})
obs_df

In [ ]:
obs_coords_df = obs_df.merge(coords_df, on="cell", how="inner")
obs_coords_df

### Merge datasets

In [ ]:
# merged_df = metabolic_task_df.merge(obs_coords_df, on="cell", how="inner") \
#                .merge(reactions_df, on="cell", how="inner") \
#                .merge(gene_df, on="cell", how="inner")

merged_df = gene_df.merge(obs_coords_df, on="cell", how="inner") \
               .merge(reactions_df, on="cell", how="inner")

# merged_df = metabolic_task_df.merge(obs_coords_df, on="cell", how="inner")
# merged_df

In [ ]:
# merged_df = flux_df.merge(obs_coords_df, on="cell", how="inner")
# merged_df

### Distribution of Cell count and status per Subject ID

In [ ]:
status_counts = merged_df.groupby(["Subject_ID", "Treatment_Status"]).size().unstack(fill_value=0)
status_counts

In [ ]:
cell_counts = merged_df.groupby("Subject_ID")["cell"].nunique()
cell_counts

In [ ]:
summary = status_counts.join(cell_counts.rename("num_cells"))
summary

## Feature Selection and Dataset Creation

In [ ]:
# nums = [str(i) for i in range(107)]
# nums
dataset = merged_df.drop(columns=["cell", "sample"])
dataset

In [ ]:
# dataset = merged_df[["M_4","M_6","M_15","Subject_ID","Treatment_Status"]]
dataset["treatment_encoded"] = dataset["Treatment_Status"].map({"Untreated": 0, "Treated": 1})
dataset

### Class Balance Analysis

In [ ]:
dataset["Treatment_Status"].value_counts()

In [ ]:
plt.figure(figsize=(5,4))
dataset["Treatment_Status"].value_counts().plot(
    kind="bar",
    color=["skyblue", "salmon"]
)
plt.title("Label Distribution (Treated vs UnTreated)")
plt.ylabel("Count")
plt.xlabel("Class")
plt.xticks(rotation=0)
plt.show()

# Classification Problem (Treated vs Untreated Cells)

## Dataset Definition

In [ ]:
X = dataset.drop(columns=["Treatment_Status","Subject_ID","treatment_encoded"])
y = dataset["treatment_encoded"]
feature_names = X.columns.tolist()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_idx, test_idx in gss.split(X, y, groups=groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [ ]:
sgkf = StratifiedGroupKFold(n_splits=5)

for train_idx, test_idx in sgkf.split(X, y, groups=groups):
    print("Train subjects:", len(set(groups[train_idx])))
    print("Test subjects:", len(set(groups[test_idx])))
    print("Train treated ratio:", (y[train_idx]==1).mean())
    print("Test treated ratio:", (y[test_idx]==1).mean())
    break

## Classic Pipeline

In [ ]:
# Standardization Dataset
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=feature_names,
    index=X_train.index
)
X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    columns=feature_names,
    index=X_test.index
)

In [ ]:
# Train Randomforest Classifier
rf = RandomForestClassifier(
   n_estimators=300, 
   random_state=42, 
   max_depth=8, 
   min_samples_split=20, 
   min_samples_leaf=10,
   n_jobs=-1
)
rf.fit(X_train_scaled, y_train)

In [ ]:
y_pred = rf.predict(X_test_scaled)

In [ ]:
print(classification_report(y_pred=y_pred, y_true=y_test))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:\n", cm)

# Display
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap="Blues")
plt.show()

In [ ]:
df_pred = pd.DataFrame({
    "subject": adata.obs.iloc[test_idx]["Subject_ID"].values,
    "true": y_test,
    "pred": y_pred
})
df_pred

In [ ]:
valid_groups = df_pred.groupby("subject").filter(lambda x: len(x) > 0)
valid_groups

In [ ]:
def majority_vote(x):
    if x.empty:
        return np.nan
    return x.mode().iloc[0]

subject_pred = df_pred.groupby("subject")["pred"].apply(majority_vote).dropna()
subject_true = df_pred.groupby("subject")["true"].first().loc[subject_pred.index]

acc_subject = (subject_pred == subject_true).mean()
print("Subject-level accuracy:", acc_subject)

In [ ]:
from sklearn.metrics import balanced_accuracy_score
print("Balanced Accuracy:", balanced_accuracy_score(subject_true, subject_pred))

## SHAP Analysis

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(X_test_scaled)

In [ ]:
sv_class1 = shap_values[:, :, 1]
shap.summary_plot(sv_class1, features=X_test_scaled, feature_names=feature_names)

In [ ]:
# Plot both of them
fig, axes = plt.subplots(1, shap_values.shape[2], figsize=(12, 6))

for c in range(shap_values.shape[2]):
    shap.summary_plot(
        shap_values[:, :, c], 
        features=X_test_scaled, 
        feature_names=feature_names, 
        show=False,  
        plot_size=None
    )
    plt.sca(axes[c])
    plt.title(f"Class {c}")

plt.tight_layout()
plt.show()

In [ ]:
# shap diff plot
shap_diff = shap_values[:, :, 1] - shap_values[:, :, 0]
shap.summary_plot(
    shap_diff, 
    features=X_test_scaled, 
    feature_names=feature_names
)

In [ ]:
# Feature importance via mean(|SHAP|)
mean_abs_shap = np.abs(shap_values[:,:,1]).mean(axis=0)
fi = pd.Series(mean_abs_shap, index=feature_names).sort_values(ascending=False)
print("\nTop features by mean(|SHAP|):\n", fi.head(10))

In [ ]:
n_classes = shap_values.shape[2]

for c in range(n_classes):
    mean_abs_shap = np.abs(shap_values[:, :, c]).mean(axis=0)
    fi = pd.Series(mean_abs_shap, index=feature_names).sort_values(ascending=False)

    plt.figure(figsize=(8, 6))
    fi.head(10).plot(kind='barh')
    plt.xlabel("Mean(|SHAP value|)")
    plt.ylabel("Features")
    # plt.title(f"Top 10 Features by SHAP (Class {c})")
    plt.title(f"Top 10 Features by SHAP (Treated Class)")
    plt.gca().invert_yaxis()
    plt.show()

# Modern Pipeline

### Preprocessing

In [ ]:
# define numerical and categorical feature title
# num_features = [col for col in X.columns if col.startswith("M")]
num_features = X.columns
# cat_features = ["Subject_ID"]

In [ ]:
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        # ("cat", categorical_transformer, cat_features)
    ])

### Define Classifier Pipeline with Preprocessor

In [ ]:
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=300, random_state=42, max_depth=8, min_samples_split=20, min_samples_leaf=10))
    # ("classifier", LogisticRegression(max_iter=1000))
])

In [ ]:
clf.fit(X_train, y_train)

### Processed X data

In [ ]:
Xt = preprocessor.fit_transform(X_train)
feature_names = preprocessor.get_feature_names_out()
# Xt_df = pd.DataFrame(Xt.toarray(), columns=list(feature_names))
Xt_df = pd.DataFrame(Xt, columns=list(feature_names))
Xt_df

In [ ]:
# for col in [c for c in Xt_df.columns if c.startswith("num")]:
#     sns.boxplot(x=y_train, y=col, data=Xt_df)
#     plt.title(f"{col} by Treatment")
#     plt.show()

### Feature Importance

In [ ]:
# Mutual Information (non-linear relationships)
mi = mutual_info_classif(Xt_df, y_train, discrete_features='auto')
mi_series = pd.Series(mi, index=Xt_df.columns).sort_values(ascending=False)
mi_series

In [ ]:
# Feature importance from a model
importances = pd.Series(clf['classifier'].feature_importances_, index=Xt_df.columns).sort_values(ascending=False)
importances

In [ ]:
# Pearson / Point-biserial correlation (continuous features vs binary target)
corr = Xt_df.corrwith(y_train)
corr.sort_values()

### Model Evaluation

In [ ]:
y_pred = clf.predict(X_test)
classification_report(y_test, y_pred)

In [ ]:
cm = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:\n", cm)

# Display
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap="Blues")
plt.show()